# NB EMA

1) organizza il notebook in funzioni diverse, di modo che puoi ripetere le analisi su più di un dataset
2) esporta in csv il nuovo dataset, magari con le analisi fatte e pusha 
3) comincia a pensare alla normalizzazione 
4) vedi se questi cazzo di dati sono block sparse oppure no 


###  DS lab 2019

## Behavioural data

In [1]:
import pandas as pd
import numpy as np
import matplotlib as mpl

In [2]:
# import data into csv file

alldata_beh = pd.read_csv('data/Behavioral/AllData.csv')

/anaconda3/envs/dslab_1/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (66,68,80,81,82,83,84,95,96,97,98,99,110,111,112,113,114,125,126,127,128,129,140,141,142,144,155,156,157,159,932,933,934,935,936,937,938,939,940,941,942,943,944,945,946,947,948,949,950,951,952,953,954,955,956,957,958,959,960,961,962,963,964,965,966,967,968,969,970,971,972,973,974,975,976,977,978,979,980,981,982,983,984,985,986,987,988,989,990,991,992,993,994,995,996,997,998,999,1000,1001,1002,1003,1004,1005,1006,1007,1008,1009,1010,1011,1012,1013,1014,1015,1016,1017,1018,1019,1020,1021,1022,1023,1024,1025,1026,1027,1028,1029,1030,1031,1032,1033,1034,1035,1036,1037,1038,1039,1040,1041,1042,1043,1044,1045,1046,1047,1048,1049,1050,1051,1052,1053,1054,1055,1056,1057,1058,1059,1060,1061,1062,1063,1064,1065,1066,1067,1068,1069,1070,1071,1072,1073,1074,1075,1076,1077,1078,1079,1080,1081,1082,1083,1084,1085,1086,1087,1088,1089,1090,1091,1092,1093,1094,1095,1096,1097,1098,1099,1100,1

### Nan percentages

In [4]:
def find_nan_percentages(dataframe): #determining NaN percentages in the dataframe, returns dict ith nan percentages
    
    nan_percentages = []    # [COLUMN_NAME, PERCENTAGE OF NANs]
    for column in dataframe: 
        n_elements = 0
        n_nan_elements = 0
        for row in dataframe.index: 
            elem = dataframe.loc[row, column]
            if pd.isna(elem)==True :
                n_nan_elements+=1
            n_elements+=1
        nan_percentages.append([column, round((n_nan_elements/n_elements),2)*100])
    nan_perc = dict(nan_percentages) #NaN percentages dict
    
    return nan_perc



In [ ]:
#isolate behavioral test data : no id info, nor diagnoses

beh_test_data = alldata_beh[ list(alldata_beh.loc[:,'ACE_01':]) ] 
beh_test_data

In [5]:
# find features within a threshold of NaN percentage, returns a list of the features

def find_frequent_features(dataframe, perc_threshold):
    nan_perc = find_nan_percentages(dataframe)
    freq_features = []   #features with less or equal than 50% NaN 
    for feature in dataframe.columns: 
        if nan_perc.get(feature) < perc_threshold: 
            freq_features.append(feature)
    return freq_features
    

#btdata_f = beh_test_data.loc[:,freq_features]
        

In [5]:
# finding basic features : id, basics, diagnoses

basic_columns = alldata_beh[ list(alldata_beh.loc[:,:'Days_Baseline'])].columns  #columns with id, basic features and diagnoses
basic_features = []       #list with basic features 
for col in basic_columns: 
    basic_features.append(col)


In [ ]:
# filter alldata_beh with only basic and frequent features

allbdata_f = alldata_beh.loc[:, (basic_features+freq_features)]  #filtered alldata
allbdata_f  



In [ ]:
# exporting in csv 

# file = pd.to_csv('path')

### Nan percentages range in dataset

In [6]:
def nan_perc_ranges(dataframe): #returns list of Nan perc ranges, with associated features
    
    nan_percentages = find_nan_percentages(dataframe)
    features_perc_range = []   #list with percentages of NaN ranges associated with features
                               # [10, list_of_features] element means that labels associated are from 10% to 19.99% of NaN 
                               # note that for the last category features with 100% NULL are included
    for threshold in np.arange(10):
        feature_list = []
        for feature, perc in nan_percentages.items():
            if threshold != 9: 
                if perc >= threshold*10.0 and perc < (threshold+1)*10.0: 
                    feature_list.append(feature)
            else: 
                if perc >= threshold*10.0 and perc <= (threshold+1)*10.0: 
                    feature_list.append(feature)
        features_perc_range.append([threshold*10,feature_list])
        
    return features_perc_range

In [ ]:
# printing NaN percentages ranges with associated features
features_perc_range = nan_perc_ranges(alldata_beh)
for perc,features in features_perc_range:
    print('number of features in', perc,'% - ', perc+10,'% range :', len(features))

## Matching columns with descriptions

In [ ]:
# reading valid column names from the description files 

import glob

path = 'data/Behavioral/description' 
desc_files = glob.glob(path + "/*.xlsx")

varname_cols = []
desc_errors = 0
wrong_desc = []
processed_files = [] #list of processed files to avoid errors

for filename in desc_files:
    df = pd.read_excel(filename, skiprows=[0]) #index_col=None, header=0
    column_names = df.columns
    #print(column_names)
    if 'Variable Name' in column_names: 
        varname_cols.append(df.loc[:,'Variable Name'])
    elif 'Variable' in column_names: 
        varname_cols.append(df.loc[:,'Variable'])
    else:
        desc_errors += 1
        wrong_desc.append(filename)
    processed_files.append(filename)

print([file for file in processed_files])
        
'''
print ('description that do not contain a column that matches are :', desc_errors ,'\n',
       'the wrong descriptions are ',
        [d for d in wrong_desc])
        
FOUND THAT 'data/Behavioral/description/ESWAN.xlsx' RAISES PROBLEMS. NAIVE CORRECTION
'''

# fix issue with ESWAN 

path_ESWAN = 'data/Behavioral/description/ESWAN.xlsx'
df = pd.read_excel(path_ESWAN, skiprows=[0,1]) #index_col=None, header=0
#column_names = df.columns
varname_cols.append(df.loc[:,'Variable Name'])

# fix issue with nested directory 

path_inside_nested_dir = 'data/Behavioral/description/*'
desc_files_nested = glob.glob(path_inside_nested_dir + "/*.xlsx")
for filepath in desc_files_nested:
    df = pd.read_excel(filepath, skiprows=[0])
    varname_cols.append(df.loc[:,'Variable Name'])

# resulting list

print ('the resulting list has lenght', len (varname_cols))  #outputs 123. works. 




In [ ]:
varnames_df = pd.DataFrame(np.hstack([i.values for i in varname_cols])) 
varnames_df = varnames_df.dropna()    #drop nan
varnames = varnames_df.values  # numpy array with variable names from descriptions
varnames_list = [elem[0] for elem in varnames.tolist()]  #list with variable names from descriptions

In [ ]:
beh_data_matched = alldata_beh.copy(deep=True)   # dataframe with behavioural data keeping only basic columns and the ones mentioned in the decriptions
for column in beh_data_matched.columns: 
    if column not in varnames_list and column not in basic_features:
        beh_data_matched.drop(columns=column, inplace=True)

In [ ]:
beh_data_matched

In [ ]:
#converting to strings as the original data is, before exporting in csv
beh_data_matched = beh_data_matched.astype(str)

In [ ]:
# exporting in csv behavioral data matched
file = beh_data_matched.to_csv('data/Behavioral/behdata_matched_ema.csv')

## Working with cleaned (behavioural) data

In [2]:
# importing clean data 

finalSummaries = pd.read_csv('data/Behavioral/cleaned/finalSummaries.csv')
finalSummariesLabels = pd.read_csv('data/Behavioral/cleaned/finalSummariesLabels.csv')
HBNFinalSummaries = pd.read_csv('data/Behavioral/cleaned/HBNFinalSummaries.csv')

/anaconda3/envs/dslab_1/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (53,83,85,97,98,99,100,101,112,113,114,115,116,127,128,129,131,142,143,144,146,263,278,279,282,283,284,287,288,289,292,293,294) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [8]:
cleandata = [finalSummaries, finalSummariesLabels, HBNFinalSummaries]

# shapes
for dataset in cleandata: 
    print(dataset.shape)
    

(326, 1)
(334, 2)
(2096, 399)


In [7]:
# why do finalSummaries, finalSummariesLabels differ in row size? 

non_matched_labels = []
for label in finalSummaries.loc[:,'x']: 
    if label not in finalSummariesLabels.loc[:,'label'].values:
        non_matched_labels.append(label)
print('number of non-matched labels: ',len(non_matched_labels),'\n')

dupli = pd.DataFrame(finalSummariesLabels.duplicated(subset=['label']))

duplicate_labels = []
for index in dupli.index[dupli[0]==True].to_list(): 
    duplicate_labels.append(finalSummariesLabels.loc[index, 'label'])

print(finalSummariesLabels[finalSummariesLabels['label'].isin(duplicate_labels)] )
    

number of non-matched labels:  0 

           label                                   question
1020   CELF_WC_R                     Word Classes Raw Score
1023   CELF_FD_R             Following Directions Raw Score
1026   CELF_FS_R             Formulated Sentences Raw Score
1029   CELF_RS_R              Recalling Sentences Raw Score
1032  CELF_USP_R  Understanding Spoken Paragraphs Raw Score
1053   CELF_WC_R                     Word Classes Raw Score
1056   CELF_FD_R             Following Directions Raw Score
1059   CELF_FS_R             Formulated Sentences Raw Score
1062   CELF_RS_R              Recalling Sentences Raw Score
1065  CELF_USP_R  Understanding Spoken Paragraphs Raw Score
5689     SWAN_IN                        Inattention Average
5690     SWAN_HY                      Hyperactivity Average
5691  SWAN_Total                               SWAN Average
5711     SWAN_IN                        Inattention Average
5712     SWAN_HY                      Hyperactivity Average
5713 

There are no labels in finalSummaries that are not in finalSummariesLabels. 

In [8]:
HBNFinalSummaries.loc[:,duplicate_labels]

/anaconda3/envs/dslab_1/lib/python3.6/site-packages/pandas/core/indexing.py:1418: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


,CELF_WC_R,CELF_FD_R,CELF_FS_R,CELF_RS_R,CELF_USP_R,SWAN_IN,SWAN_HY,SWAN_Total
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
2092,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2093,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2094,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2095,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
# nan percentages in HBNSummaries

HBN_nanpercranges = nan_perc_ranges(HBNFinalSummaries)

# printing NaN percentages ranges with associated features
for perc,features in HBN_nanpercranges:
    print('number of features in', perc,'% - ', perc+10,'% range :', len(features))

number of features in 0 % -  10 % range : 13
number of features in 10 % -  20 % range : 41
number of features in 20 % -  30 % range : 40
number of features in 30 % -  40 % range : 33
number of features in 40 % -  50 % range : 15
number of features in 50 % -  60 % range : 8
number of features in 60 % -  70 % range : 16
number of features in 70 % -  80 % range : 17
number of features in 80 % -  90 % range : 27
number of features in 90 % -  100 % range : 189


In [12]:
#discarding features with >= 90.0 percentage of Nan
HBN_reduced = HBNFinalSummaries.loc[:,find_frequent_features(HBNFinalSummaries, 90.0)] # HBNFinalSummaries reduced to columns with less than 90% Nan

In [14]:
#are there binary variables? 

non_binary_variables = []

for column in HBN_reduced.columns: 
    for row in HBN_reduced.index: 
        elem = HBN_reduced.loc[row, column]
        if elem != 0.0 and elem != 1.0:
            non_binary_variables.append(column)
            break


for column in HBN_reduced.columns: 
    if column not in non_binary_variables: 
        print(column)

Sex


In [ ]:
HBN_rc_labels = finalSummariesLabels[finalSummariesLabels['label'].isin(HBN_reduced.columns)]

'''
for row in HBN_rc_labels.index: 
    print('label : ',HBN_rc_labels.loc[row,'label'],'\t question : ',HBN_rc_labels.loc[row,'question'],'\n')
'''

In [ ]:
with open('/Users/Proprietario/Desktop/HBN_rc_labels.txt', 'w') as wfile: 
    for row in HBN_rc_labels.index: 
        wfile.write('label : '+HBN_rc_labels.loc[row,'label']+'\t question : '+HBN_rc_labels.loc[row,'question']+'\n')

In [16]:
HBN_rc_labels['label']

189           APQ_P_Total
190             APQ_P_OPD
249            APQ_SR_OPD
250          APQ_SR_Total
259     ARI_P_Total_Score
              ...        
6655              YSR_RBB
6657               YSR_SC
6663              YSR_Ext
6665              YSR_Int
6669            YSR_Total
Name: label, Length: 171, dtype: object

In [ ]:
for column in HBN_reduced.columns: 
    if column not in HBN_rc_labels['label'].to_list():
        print(column)

In [ ]:
HBN_reduced.columns.to_list()

## Working with HBN_reduced

In [17]:
#isolate diagnoses labels 
HBNrc_df_diagnoses = HBN_reduced.loc[:,'NoDX':'DX_10_Time']
HBNrc_labels_diagnoses = HBNrc_df_diagnoses.columns.to_list()

In [18]:
#isolate basic labels
HBNrc_df_basic = HBN_reduced.loc[:,
['Anonymized.ID',
 'EID',
 'Sex',
 'Age',
 'Study.Site']]
HBNrc_labels_basic = HBNrc_df_basic.columns.to_list()

In [19]:
HBNrc_labels_basic

['Anonymized.ID', 'EID', 'Sex', 'Age', 'Study.Site']

In [20]:
#isolate bvr labels, 'bvr' = behavioural data from summaries
HBNrc_df_bvr = HBN_reduced.loc[:, 'APQ_P_OPD':]
HBNrc_labels_bvr = HBNrc_df_bvr.columns.to_list()

In [21]:
#is this dataset block sparse?
ranges_list = nan_perc_ranges(HBNrc_df_bvr)

In [22]:
ranges_list.pop()

[90, []]

In [23]:
ranges_list

[[0, ['EHQ_Total']],
 [10,
  ['APQ_P_OPD',
   'APQ_P_Total',
   'APQ_SR_OPD',
   'APQ_SR_Total',
   'ARI_P_Total_Score',
   'ARI_S_Total_Score',
   'ASSQ_Total',
   'Barratt_Total_Edu',
   'Barratt_Total_Occ',
   'Barratt_Total',
   'CBCL_AD',
   'CBCL_WD',
   'CBCL_SC',
   'CBCL_RBB',
   'CBCL_AB',
   'CBCL_Int',
   'CBCL_Ext',
   'CBCL_C',
   'CBCL_Total',
   'CELF_Total',
   'SCQ_Total',
   'SRS_AWR',
   'SRS_COG',
   'SRS_COM',
   'SRS_DSMRRB',
   'SRS_MOT',
   'SRS_RRB',
   'SRS_SCI',
   'SRS_Total',
   'WIAT_Num_Raw',
   'WIAT_Spell_Raw']],
 [20,
  ['CTOPP_EL_R',
   'CTOPP_BW_R',
   'CTOPP_NR_R',
   'CTOPP_RD_R',
   'CTOPP_RL_R',
   'CTOPP_RSN_Comp',
   'DTS_absorption',
   'DTS_appraisal',
   'DTS_regulation',
   'DTS_tolerance',
   'DTS_total',
   'ICU_P_Callous',
   'ICU_P_Uncaring',
   'ICU_P_Unemotional',
   'ICU_P_Total',
   'NLES_P_TotalEvents',
   'NLES_P_Upset_Total',
   'PCIAT_Total',
   'mdx',
   'SCARED_P_GD',
   'SCARED_P_PN',
   'SCARED_P_SC',
   'SCARED_P_SH',
   '

In [ ]:
for row in HBN_reduced.index: 
    print(HBN_reduced.loc[row,['WISC_BD_Raw',
     'WISC_Similarities_Raw',
     'WISC_MR_Raw',
     'WISC_DS_Raw',
     'WISC_Coding_Raw',
     'WISC_Vocab_Raw',
     'WISC_FW_Raw',
     'WISC_VP_Raw',
     'WISC_PS_Raw',
     'WISC_SS_Raw',
     'WISC_VSI',
     'WISC_VCI',
     'WISC_FRI',
     'WISC_WMI',
     'WISC_PSI',
     'WISC_FSIQ',]])

In [ ]:
'''

# from the interet (pdf from pearsonclinical.com) I found that normal values fro WASI test can be 

label : WASI_Matrix_Raw	 question : Matrix Reasoning Raw Score  	 avg : 21
label : WASI_Sim_Raw	 question : Similarities Raw Score  	 avg : 32
label : WASI_PRI_Comp	 question : PRI Composite Score 	 avg : 100
label : WASI_VCI_Comp	 question : VCI Composite Score 	 avg : 100
label : WASI_Vocab_Raw	 question : Vocab Raw Score 	 avg : 38
label : WASI_BD_Raw	 question : Block Design Raw Score 	 avg : 44
            
'''      


In [15]:
HBNFinalSummaries.loc[:,'SWAN_HY']

KeyError: 'SWAN_HY'